In [1]:
import numpy as np
import pandas as pd

In [2]:
cen = pd.read_csv('../data/processed/censimento.csv')
mc = pd.read_csv('../data/processed/massa_critica.csv')
ssd = pd.read_excel('../data/SSD_V3_area.xlsx')

In [3]:
cen.head(3)

,Nome,Cognome,Codice fiscale,SSD,Email,age
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65


In [4]:
mc.head(3)

,Nome,Cognome,Codice fiscale,Inquadramento contrattuale,MC
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,Ricercatore,True
1,Tiziana,Tesauro,TSRTZN71H58F205V,Ricercatore (III livello),True
2,Pietro,Siciliano,SCLPRL60D26I549H,"Dirigente di Ricerca, I Livello",True


In [5]:
ssd.head()

,Cod. SSD,Area,Area_desc
0,MATH-01/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
1,MATH-01/B,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
2,MATH-02/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
3,MATH-02/B,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
4,MATH-03/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE


In [6]:
df = pd.merge(cen, mc[['Codice fiscale','Inquadramento contrattuale',
       'MC' ]], on='Codice fiscale', how='left')

In [7]:
df['MC'].fillna(False, inplace=True)
df['Inquadramento contrattuale'].fillna(np.nan, inplace=True)

/var/folders/zh/3166vnf95tnblssyvp6rbhyc0000gn/T/ipykernel_9068/3115992713.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['MC'].fillna(False, inplace=True)
/var/folders/zh/3166vnf95tnblssyvp6rbhyc0000gn/T/ipykernel_9068/3115992713.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['MC'].fillna(False

In [8]:
df.MC.value_counts()

MC
False    471
True     232
Name: count, dtype: int64

In [9]:
import re
import requests
from bs4 import BeautifulSoup


DOCENTI_URL_DEFAULT = (
    "https://cercauniversita.mur.gov.it/php5/docenti/vis_docenti.php?docinput={}&docsubmit=cerca"
)
ASSEGNASTI_URL_DEFAULT = (
    "https://cercauniversita.mur.gov.it/php5/assegnisti/vis_assegnisti.php?"
    "qualifica=**&argomento=&title_radiogroup=P&cognome={}&nome={}&radiogroup=E&universita=00&facolta=00&"
    "settore=0000&area=0000&situazione_al=0&vai=Invio"
)


def fetch_ssd(
    first_name: str,
    last_name: str,
    *,
    timeout: int = 10,
    session: requests.Session | None = None,
    docenti_url_template: str = DOCENTI_URL_DEFAULT,
    assegnisti_url_template: str = ASSEGNASTI_URL_DEFAULT,
) -> str:
    """
    Fetch SSD by querying first DOCENTI (by last_name), then ASSEGNASTI (by last_name + first_name).

    Returns:
        - SSD string if found
        - "NULL" otherwise
    """

    def _extract_ssd_from_url(url: str) -> str:
        try:
            s = session or requests
            resp = s.get(url, timeout=timeout)
            resp.raise_for_status()
        except requests.RequestException:
            return "NULL"

        soup = BeautifulSoup(resp.text, "html.parser")
        table = soup.find("table", {"class": "risultati"})
        if not table:
            return "NULL"

        rows = table.find_all("tr")[1:]  # skip header row
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 7:
                ssd_2024 = cols[5].get_text(strip=True)
                department = cols[6].get_text(strip=True)

                # Your original logic: if SSD looks like a department name (some assegnisti), return department
                if re.search(r"\(\w+\)", ssd_2024):
                    return department

                return ssd_2024

        return "NULL"

    # Normalise inputs a bit
    first_name = (first_name or "").strip()
    last_name = (last_name or "").strip()

    if not last_name:
        return "NULL"

    # 1) DOCENTI lookup (only last name in your current URL design)
    ssd_value = _extract_ssd_from_url(docenti_url_template.format(last_name))

    # 2) ASSEGNASTI fallback (requires both last + first name)
    if ssd_value == "NULL" and first_name:
        ssd_value = _extract_ssd_from_url(assegnisti_url_template.format(last_name, first_name))

    return ssd_value


import requests

# Reuse one session (faster + nicer to the remote server)
sess = requests.Session()

col = "SSD"  

mask = df[col].isna() | (df[col].astype(str).str.strip().str.upper() == "NULL")

df.loc[mask, col] = df.loc[mask].apply(
    lambda row: fetch_ssd(row["Nome"], row["Cognome"], session=sess),
    axis=1,
)

In [10]:
df[df['SSD'].isna()]

,Nome,Cognome,Codice fiscale,SSD,Email,age,Inquadramento contrattuale,MC


In [11]:
df['SSD'] = df['SSD'].apply(lambda x : x.strip().upper())

In [12]:
df

,Nome,Cognome,Codice fiscale,SSD,Email,age,Inquadramento contrattuale,MC
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53,Ricercatore,True
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46,Dirigente di Ricerca,True
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65,I° Ricercatore TI,True
3,Paolo,Barsocchi,BRSPLA78S27G702C,INFO-01/A,paolo.barsocchi@isti.cnr.it,48,Primo ricercatore,True
4,Antonino,Colanzi,CLNNNN64B29E435X,BIOS-10/A,antonino.colanzi@cnr.it,62,"II Livello, fascia 3",True
...,...,...,...,...,...,...,...,...
698,Francesco Paolo,Fanizzi,FNZFNC56S07C975L,CHEM-03/A,francesco.fanizzi@unisalento.it,70,NaN,False
699,Biagio,Iacolare,CLRBGI97M15F839A,STAT-03/A,biagio.iacolare@unina.it,29,NaN,False
700,Marta,Massei,MSSMRT93A69G843T,PSIC-01/B,marta.massei@unifi.it,33,NaN,False
701,Venturina,Stagnu,STGVTR77P46H501I,BIOS-14/A,venturina.stagni@cnr.it,49,NaN,False


In [13]:
total_censiti = len(df)
mc = sum(df['MC'])
ricercatori = total_censiti - mc

In [14]:
df['SSD'].value_counts()

SSD
STAT-03/A     36
MEDS-05/A     29
GSPS-05/A     21
MEDS-02/A     21
IINF-05/A     20
              ..
MEDS-26/B      1
MEDS-25/B      1
CHEM-01/A      1
STEC-01/A      1
09/IINF-05     1
Name: count, Length: 165, dtype: int64

In [15]:
df

,Nome,Cognome,Codice fiscale,SSD,Email,age,Inquadramento contrattuale,MC
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53,Ricercatore,True
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46,Dirigente di Ricerca,True
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65,I° Ricercatore TI,True
3,Paolo,Barsocchi,BRSPLA78S27G702C,INFO-01/A,paolo.barsocchi@isti.cnr.it,48,Primo ricercatore,True
4,Antonino,Colanzi,CLNNNN64B29E435X,BIOS-10/A,antonino.colanzi@cnr.it,62,"II Livello, fascia 3",True
...,...,...,...,...,...,...,...,...
698,Francesco Paolo,Fanizzi,FNZFNC56S07C975L,CHEM-03/A,francesco.fanizzi@unisalento.it,70,NaN,False
699,Biagio,Iacolare,CLRBGI97M15F839A,STAT-03/A,biagio.iacolare@unina.it,29,NaN,False
700,Marta,Massei,MSSMRT93A69G843T,PSIC-01/B,marta.massei@unifi.it,33,NaN,False
701,Venturina,Stagnu,STGVTR77P46H501I,BIOS-14/A,venturina.stagni@cnr.it,49,NaN,False


In [16]:
cnt = 0
bad_CF = []
for i, ss in enumerate(df['SSD']):
    if ss not in ssd['Cod. SSD'].tolist():
        bad_CF.append(df['Codice fiscale'].tolist()[i])
        cnt += 1
        try:
            print(ss.replace(' ', '*'))
        except:
            print(f'--------------->', ss)
print(cnt)

05/BIOS-08
MED/26*–*NEUROLOGIA
BIO/12*BIOCHIMICA*CLINICA*E*BIOLOGIA*MOLECOLARE*CLINICA
M-EDF/01
SECS-P02;*SECS-P01
STATISTICA*SOCIALE*SECS*05
GLOT-01/A
INF-04/A
COMP*-01/A
BIO/18*GENETICA
ING-INF/05
BIOS/07A
05/BIOS-06
MEDS*02/A
PHYS-03/A;
PHYS-06/A;*CHEM-01/A
GSPS-05/A*-*SOCIOLOGIA*GENERALE.
06/MEDS-02*06/A2*-*PATOLOGIA
GENERALE*E*PATOLOGIA
CLINICA
03/CHEM-07-A*-*CHIMICA*FARMACEUTICA
MED/09
13/STAT-03
STAT/03B
MEDS/24-A
STAT/03B
STAT/03B
STAT/02
BIOS10/A
SECS-S/04*–*DEMOGRAFIA,*SECS-S/05*–*STATISTICA*SOCIALE
13/ECON-01
GERONTOLOGIA
GSPS-05/A*–*SOCIOLOGIA*GENERALE
M-FIL/03
PHYS-03/A*;*CHEM-01/A
MED/01
14/GSPS-05
MEDS-5/A
MEDS-5/A
BIOS-9/A
BIO/18*GENETICA
06/MEDS-26
SECS-S/05*–*STATISTICA*SOCIALE
MEDS-26D
GSPS-05/A*(GENERAL*SOCIOLOGY)
M-EDF/01
BIOS-06
02/PHYS-06
06/MEDS-10
NULL
10/ITAL-01
CEAR/08*C*-*TECHNOLOGICAL*AND*ENVIRONMENTAL*DESIGN*OF*ARCHITECTURE
PHYS-03/A*;*CHEM-01/A
PHYS-03/A*;*CHEM-01/A
BIO/12*BIOCHIMICA*CLINICA*E*BIOLOGIA*MOLECOLARE*CLINICA
MED-18
ING-INF*05
05/BIOS-10
09/II

In [18]:
df[df['Codice fiscale'].isin(bad_CF)].drop_duplicates(subset=['Codice fiscale']).to_excel('bad_SSD.xlsx', index=False)

In [45]:
df[df['SSD'] == 'CLINICA']

,Nome,Cognome,Codice fiscale,SSD,Email,age,Inquadramento contrattuale,MC,Cod. SSD,Area,Area_desc


In [38]:
df.loc[~df['SSD'].isin(ssd['Cod. SSD']), 'SSD'] = "NA"

In [39]:
df

,Nome,Cognome,Codice fiscale,SSD,Email,age,Inquadramento contrattuale,MC
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53,Ricercatore,True
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46,Dirigente di Ricerca,True
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65,I° Ricercatore TI,True
3,Paolo,Barsocchi,BRSPLA78S27G702C,INFO-01/A,paolo.barsocchi@isti.cnr.it,48,Primo ricercatore,True
4,Antonino,Colanzi,CLNNNN64B29E435X,BIOS-10/A,antonino.colanzi@cnr.it,62,"II Livello, fascia 3",True
...,...,...,...,...,...,...,...,...
698,Francesco Paolo,Fanizzi,FNZFNC56S07C975L,CHEM-03/A,francesco.fanizzi@unisalento.it,70,NaN,False
699,Biagio,Iacolare,CLRBGI97M15F839A,STAT-03/A,biagio.iacolare@unina.it,29,NaN,False
700,Marta,Massei,MSSMRT93A69G843T,PSIC-01/B,marta.massei@unifi.it,33,NaN,False
701,Venturina,Stagnu,STGVTR77P46H501I,BIOS-14/A,venturina.stagni@cnr.it,49,NaN,False


In [40]:
df = pd.merge(df, ssd, right_on='Cod. SSD', left_on='SSD', how='left')

In [19]:
df

,Nome,Cognome,Codice fiscale,SSD,Email,age,Inquadramento contrattuale,MC
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53,Ricercatore,True
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46,Dirigente di Ricerca,True
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65,I° Ricercatore TI,True
3,Paolo,Barsocchi,BRSPLA78S27G702C,INFO-01/A,paolo.barsocchi@isti.cnr.it,48,Primo ricercatore,True
4,Antonino,Colanzi,CLNNNN64B29E435X,BIOS-10/A,antonino.colanzi@cnr.it,62,"II Livello, fascia 3",True
...,...,...,...,...,...,...,...,...
698,Francesco Paolo,Fanizzi,FNZFNC56S07C975L,CHEM-03/A,francesco.fanizzi@unisalento.it,70,NaN,False
699,Biagio,Iacolare,CLRBGI97M15F839A,STAT-03/A,biagio.iacolare@unina.it,29,NaN,False
700,Marta,Massei,MSSMRT93A69G843T,PSIC-01/B,marta.massei@unifi.it,33,NaN,False
701,Venturina,Stagnu,STGVTR77P46H501I,BIOS-14/A,venturina.stagni@cnr.it,49,NaN,False


In [27]:
inq = pd.read_excel('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/processed/inquadramento.xlsx')

In [29]:
df = pd.merge(df, inq, on='Inquadramento contrattuale', how='left')
df.drop(['Inquadramento contrattuale'], axis=1, inplace=True)

In [33]:
df.head(3)

,Nome,Cognome,Codice fiscale,SSD,Email,age,MC,inquadramento_clean
0,Daniela,Trisciuoglio,TRSDNL73R55H501N,BIOS-10/A,daniela.trisciuoglio@cnr.it,53,True,Ricercatore
1,Serena,Sanna,SNNSRN80S55H856X,BIOS-04/A,serena.sanna@cnr.it,46,True,Dirigente
2,Francesca,Cavalcanti,CVLFNC61T56D086E,BIOS-11/A,francesca.cavalcanti@irib.cnr.it,65,True,Ricercatore


In [36]:
ssd

,Cod. SSD,Area,Area_desc
0,MATH-01/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
1,MATH-01/B,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
2,MATH-02/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
3,MATH-02/B,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
4,MATH-03/A,AREA 01,SCIENZE MATEMATICHE E INFORMATICHE
...,...,...,...
322,GSPS-06/A,AREA 14,SCIENZE POLITICHE E SOCIALI
323,GSPS-07/A,AREA 14,SCIENZE POLITICHE E SOCIALI
324,GSPS-07/B,AREA 14,SCIENZE POLITICHE E SOCIALI
325,GSPS-08/A,AREA 14,SCIENZE POLITICHE E SOCIALI


In [39]:
df = pd.merge(df, ssd, left_on='SSD', right_on='Cod. SSD', how='left')

In [40]:
len(df)

703

In [41]:
len(df[df['MC'] == True]) #massa critica

232

In [42]:
len(df[df['MC'] == False]) # others

471

In [43]:
df['inquadramento_clean'].value_counts()

inquadramento_clean
Professore Ordinario    94
Professore Associato    74
Ricercatore             35
Dirigente               25
Admin.                   2
Direttore                2
Name: count, dtype: int64

In [44]:
df['inquadramento_clean'].value_counts(normalize=True)

inquadramento_clean
Professore Ordinario    0.405172
Professore Associato    0.318966
Ricercatore             0.150862
Dirigente               0.107759
Admin.                  0.008621
Direttore               0.008621
Name: proportion, dtype: float64

In [46]:
df['Area_desc'].value_counts()

Area_desc
SCIENZE MEDICHE                                                        172
SCIENZE ECONOMICHE E STATISTICHE                                       125
SCIENZE BIOLOGICHE                                                     100
INGEGNERIA INDUSTRIALE E DELL'INFORMAZIONE                              64
SCIENZE STORICHE, FILOSOFICHE, PEDAGOGICHE E PSICOLOGICHE               60
SCIENZE POLITICHE E SOCIALI                                             56
SCIENZE GIURIDICHE                                                      22
 SCIENZE MATEMATICHE E INFORMATICHE                                     20
SCIENZE CHIMICHE                                                        11
INGEGNERIA CIVILE E ARCHITETTURA                                        10
SCIENZE FISICHE                                                          6
SCIENZE DELL’ANTICHITÀ, FILOLOGICO-LETTERARIE E STORICO-ARTISTICHE       1
Name: count, dtype: int64

In [47]:
df['Area_desc'].value_counts(normalize=True)

Area_desc
SCIENZE MEDICHE                                                        0.265842
SCIENZE ECONOMICHE E STATISTICHE                                       0.193199
SCIENZE BIOLOGICHE                                                     0.154560
INGEGNERIA INDUSTRIALE E DELL'INFORMAZIONE                             0.098918
SCIENZE STORICHE, FILOSOFICHE, PEDAGOGICHE E PSICOLOGICHE              0.092736
SCIENZE POLITICHE E SOCIALI                                            0.086553
SCIENZE GIURIDICHE                                                     0.034003
 SCIENZE MATEMATICHE E INFORMATICHE                                    0.030912
SCIENZE CHIMICHE                                                       0.017002
INGEGNERIA CIVILE E ARCHITETTURA                                       0.015456
SCIENZE FISICHE                                                        0.009274
SCIENZE DELL’ANTICHITÀ, FILOLOGICO-LETTERARIE E STORICO-ARTISTICHE     0.001546
Name: proportion, dtype: float